In [ ]:
import os
import sys
import json
import datetime

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
d = glhe.globals.functions.load_json('EWT_MFRTRT_input.json')
d['ground-heat-exchanger'][0]['g-function-path'] = join(cwd, 'EWT_experimental_g_functions.csv')
d['ground-heat-exchanger'][0]['load-aggregation']['method'] = 'none'
d['simulation']['time-steps-per-hour'] = 60
d['simulation']['runtime'] = 14440

In [ ]:
with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in.json').simulate()

In [ ]:
df = pd.read_csv('out.csv', index_col='Date/Time', parse_dates=True)

In [ ]:
h = d['borehole-definitions'][0]['depth']
alpha = d['soil']['conductivity'] / (d['soil']['density'] * d['soil']['specific-heat'])
ts = h**2 / (9*alpha)

In [ ]:
df[['GroundHeatExchanger:GHE 1:Inlet Temp. [C]',
   'GroundHeatExchanger:GHE 1:Outlet Temp [C]']].plot(grid=True)

In [ ]:
plt.semilogx(df['Elapsed Time [s]'].values / ts, df['GroundHeatExchanger:GHE 1:Inlet Temp. [C]'].values, label='Inlet')
plt.semilogx(df['Elapsed Time [s]'].values / ts, df['GroundHeatExchanger:GHE 1:Outlet Temp [C]'].values, label='Outlet')
plt.ylabel('Temperature [C]')
plt.xlabel('t/ts')
plt.xlim(1e-8, 1e-3)
plt.legend()
plt.show()

In [ ]:
data_path = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_loads.csv'))
df_raw_data = pd.read_csv(data_path, parse_dates=True, index_col='Date/Time')

In [ ]:
df_raw_data.head(1)

In [ ]:
df_raw_data.columns

In [ ]:
df_sim_input_data = df_raw_data[['Inst. HT [1] [W]', 'mdot [kg/s]', 'Outlet 1 [C]', 'Inlet 1 [C]']].copy(deep=True)
df_sim_input_data.rename(columns = {'Inst. HT [1] [W]': 'HT Rate [W]', 
                                    'Outlet 1 [C]': 'Exp. Inlet [C]', 
                                    'Inlet 1 [C]': 'Exp. Outlet [C]'}, inplace=True)
df_sim_input_data.resample('1T').mean()
df_sim_input_data.to_csv(join(cwd, 'input_data.csv'))
df_sim_input_data.head(1)

In [ ]:
d['temperature-profile'] = [{'name': 'EWT Data', 
                             'temperature-profile-type': 'external',
                             'path': join(cwd, 'input_data.csv')}]

d['topology']['demand-side'][1] = {'comp-type': 'temperature-profile',
                                   'name': 'EWT Data'}

In [ ]:
with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in.json').simulate()

In [ ]:
df = pd.read_csv('out.csv', index_col='Date/Time', parse_dates=True)

In [ ]:
df.columns

In [ ]:
df[['GroundHeatExchanger:GHE 1:Inlet Temp. [C]',
   'GroundHeatExchanger:GHE 1:Outlet Temp [C]']].plot(grid=True)

In [ ]:
df_sim_input_data['time'] = pd.to_timedelta(df_sim_input_data.index)
df_sim_input_data['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_sim_input_data['time'] - df_sim_input_data['time'][0])
df_sim_input_data.set_index('time', inplace=True)
df_sim_input_data.index.rename('Date/Time', inplace=True)
df_sim_input_data.head(1)

In [ ]:
ax = df_sim_input_data[['Exp. Inlet [C]', 'Exp. Outlet [C]']].loc[df.index[0] : df.index[-1]].plot()
df[['GroundHeatExchanger:GHE 1:Inlet Temp. [C]',
    'GroundHeatExchanger:GHE 1:Outlet Temp [C]']].rename(columns = {'GroundHeatExchanger:GHE 1:Inlet Temp. [C]': 'Sim. Inlet [C]',
                                                                    'GroundHeatExchanger:GHE 1:Outlet Temp [C]': 'Sim. Outlet [C]'}).plot(grid=True, ax=ax, style='--')

In [ ]:
ax = df_sim_input_data[['HT Rate [W]']].rename(columns={'HT Rate [W]': 'Exp. HT Rate [W]'}).loc[df.index[0] : df.index[-1]].plot()
df[['GroundHeatExchanger:GHE 1:Heat Rate [W]']].rename(columns={'GroundHeatExchanger:GHE 1:Heat Rate [W]': 'Sim. HT Rate [W]'}).plot(grid=True, ax=ax)